In [1]:
import spacy
import pandas as pd
nlp = spacy.load("es_core_news_sm")
from gensim import corpora
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
import spacy
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:


# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
stop_words=""

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [3]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))

In [4]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [5]:
df = pd.read_csv('tweets_public.csv', encoding='utf-8', index_col='tweet_id')
df.head(5)

,airline_sentiment,is_reply,reply_count,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
tweet_id,,,,,,,,,
926419989107798016,neutral,False,0,0,Trabajar en #Ryanair como #TMA: https://t.co/r...,NaN,Fri Nov 03 12:05:12 +0000 2017,NaN,Madrid
934854385577943041,neutral,True,0,0,@Iberia @FIONAFERRER Cuando gusten en Cancún s...,NaN,Sun Nov 26 18:40:28 +0000 2017,NaN,Mexico City
945318406441635840,negative,False,0,0,Sabiais que @Iberia te trata muy bien en santi...,NaN,Mon Dec 25 15:40:45 +0000 2017,NaN,Madrid
927540721296568320,negative,False,0,0,NUNCA NUNCA NUNCA pidáis el café de Ryanair.\n...,NaN,Mon Nov 06 14:18:35 +0000 2017,NaN,Pacific Time (US & Canada)
947965901332197376,positive,True,0,0,@cris_tortu @dakar @Iberia @Mitsubishi_ES @BFG...,NaN,Mon Jan 01 23:00:57 +0000 2018,NaN,Buenos Aires


In [6]:
df = df.replace({"negative": -1, "neutral": 0, "positive":1})

In [7]:
X = df['text'] # the features we want to analyze
ylabels = df['airline_sentiment'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.5)

In [8]:
# Logistic Regression Classifier
modelLR = LogisticRegression()

pipe = Pipeline([('preprocessing', tfidf_vector),
                 ('regression-ML', modelLR)])
# model generation
pipe.fit(X_train,y_train)

Pipeline(steps=[('preprocessing',
                 TfidfVectorizer(tokenizer=<function spacy_tokenizer at 0x000001B7F41BB4C0>)),
                ('regression-ML', LogisticRegression())])

In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
# Predicting with a test dataset
predicted = pipe.predict(X_test)
#print(predicted)

# Model Accuracy
print("Logistic Regression Accuracy:",accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",precision_score(y_test, predicted, average = None))
print("Logistic Regression Recall:",recall_score(y_test, predicted,average = None))

Logistic Regression Accuracy: 0.5945602440264361
Logistic Regression Precision: [0.62679055 0.52453988 0.55495979]
Logistic Regression Recall: [0.86208733 0.38658628 0.28395062]


In [13]:
df.groupby(['airline_sentiment'])["airline_sentiment"].count()

airline_sentiment
-1    3769
 0    2609
 1    1489
Name: airline_sentiment, dtype: int64

In [40]:
predicted

array([ 0,  0, -1, ..., -1,  0,  0], dtype=int64)

In [14]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()

pipe = Pipeline([('preprocessing', tfidf_vector),
                 ('clasificador', clf)])
# model generation
pipe = pipe.fit(X_train,y_train)

predicted = pipe.predict(X_test)

In [15]:
print("Accuracy:",accuracy_score(y_test, predicted))
print("Precision:",precision_score(y_test, predicted, average = None))
print("Recall:",recall_score(y_test, predicted,average = None))

 Accuracy: 0.48398576512455516
Precision: [0.60683297 0.39852399 0.33333333]
Recall: [0.59584665 0.40693293 0.33607682]


In [16]:
from sklearn.ensemble import ExtraTreesClassifier
clf = ExtraTreesClassifier(n_estimators=100, random_state=1)
pipe = Pipeline([('preprocessing', tfidf_vector),
                 ('clasificador', clf)])
# model generation
pipe = pipe.fit(X_train,y_train)

predicted = pipe.predict(X_test)

In [17]:
print("Accuracy:",accuracy_score(y_test, predicted))
print("Precision:",precision_score(y_test, predicted, average = None))
print("Recall:",recall_score(y_test, predicted,average = None))

Accuracy: 0.5704117946110828
Precision: [0.60716953 0.50457666 0.46335079]
Recall: [0.8658147  0.33232856 0.24279835]
